In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

# Function to get the list of S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    if response.status_code != 200:
        print("Failed to retrieve S&P 500 tickers")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.strip()
        tickers.append(ticker)
    
    return tickers

# Function to get mean return and volatility
def get_data_and_stats(ticker):
    try:
        data = yf.download(ticker, start="2014-01-01", end="2024-12-10")
        if data.empty:
            print(f"No historical data found for {ticker}")
            return None, None
        data['Daily Return'] = data['Adj Close'].pct_change()
        mean_return = data['Daily Return'].mean()
        volatility = data['Daily Return'].std()
        return mean_return, volatility
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")
        return None, None

# Function to perform Monte Carlo simulation
# T = number of days to predict
# N =  
def monte_carlo_simulation(S0, mean_return, volatility, T=5, N=1000):
    results = []
    for _ in range(N):
        price = S0
        prices = []
        for _ in range(T):
            price *= np.exp((mean_return - 0.5 * volatility**2) + volatility * np.random.normal())
            prices.append(price)
        results.append(prices[-1])
    mean_price = np.mean(results)
    median_price = np.median(results)
    std_dev = np.std(results)
    return mean_price, median_price, std_dev

# Get the list of S&P 500 tickers
tickers = get_sp500_tickers()
#tickers = ['IQV']
# DataFrame to store results
results_df = pd.DataFrame(columns=['Ticker', 'Mean_Price', 'Median_Price', 'Std_Dev'])

# Run analysis for each ticker
for ticker in tickers:
    mean_return, volatility = get_data_and_stats(ticker)
    if mean_return is not None and volatility is not None:
        history = yf.Ticker(ticker).history(period='1d')
        if not history.empty and 'Close' in history.columns:
            try:
                S0 = history['Close'].iloc[0]  # Current stock price
                mean_price, median_price, std_dev = monte_carlo_simulation(S0, mean_return, volatility)
                new_row = pd.DataFrame({'Ticker': [ticker], 'Mean_Price': [mean_price], 'Median_Price': [median_price], 'Std_Dev': [std_dev]})
                results_df = pd.concat([results_df, new_row], ignore_index=True)
            except IndexError:
                print(f"Error accessing close price for {ticker}")
        else:
            print(f"No data available for {ticker}")
    else:
        print(f"Skipping {ticker} due to data retrieval issue")

print(results_df)

# Plot results
results_df.set_index('Ticker').plot(kind='bar', y=['Mean_Price', 'Median_Price', 'Std_Dev'], subplots=True, layout=(3, 1), figsize=(10, 8))
plt.tight_layout()
plt.show()


[*********************100%***********************]  1 of 1 completed
C:\Users\codym\AppData\Local\Temp\ipykernel_6948\308442678.py:74: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[********************

No historical data found for BRK.B
Skipping BRK.B due to data retrieval issue


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

No historical data found for BF.B
Skipping BF.B due to data retrieval issue


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
# Calculate expected gain over one year
def calculate_expected_gain(df):
    df['Current_Price'] = 0.0
    df['Expected_Gain'] = 0.0

    for index, row in df.iterrows():
        ticker = row['Ticker']
        current_price = yf.Ticker(ticker).history(period='1d')['Close'].iloc[0]
        df.at[index, 'Current_Price'] = current_price
        df.at[index, 'Expected_Gain'] = ((row['Mean_Price'] - current_price) / current_price) * 100
        df.at[index, '+2stdvExpectedGain'] = ((row['Mean_Price'] + row['Std_Dev'] + row['Std_Dev'] - current_price) / current_price) * 100
        df.at[index, '-2stdvExpectedLoss'] = ((row['Mean_Price'] - row['Std_Dev'] - row['Std_Dev'] - current_price) / current_price) * 100
        df.at[index, 'Coefficient of Return'] = (row['Std_Dev']/row['Mean_Price']) * 100
    return df

# Apply the function to the results DataFrame
results_df = calculate_expected_gain(results_df)

print(results_df)

# Plot results including expected gain
results_df.set_index('Ticker').plot(kind='bar', y=['Mean_Price', 'Median_Price', 'Std_Dev', 'Expected_Gain'], subplots=True, layout=(4, 1), figsize=(10, 10))
plt.tight_layout()
plt.show()


In [ ]:
# Sort results_df by Expected_Gain in descending order
results_df_sorted = results_df.sort_values(by='Expected_Gain', ascending=False)

# Print sorted results_df
print(results_df_sorted)

In [ ]:
import pandas as pd

# Assuming results_df is your DataFrame containing the results

# Specify the file path where you want to save the CSV file
csv_file_path = 'MonteCarlo.csv'

# Save DataFrame to CSV
results_df_sorted.to_csv(csv_file_path, index=False)

print(f"DataFrame successfully saved to {csv_file_path}")

In [ ]:
results_df_sorted

In [ ]:
filtered_df = results_df_sorted[
    (results_df_sorted['Expected_Gain'] > 1) & 
    (results_df_sorted['Coefficient of Return'] < 10)
]
filtered_df

# Given values
risk_free_rate_annual = 3.41 / 100  # Convert to decimal
daily_risk_free_rate = risk_free_rate_annual / 252

# Calculate the 5-day adjusted risk-free return
risk_free_return_5_days = (1 + daily_risk_free_rate) ** 1 - 1

# Function to calculate Sortino Ratio
def calculate_sortino(row):
    expected_gain = row['Expected_Gain']
    std_dev = row['Std_Dev'] / 100  # Assuming Std_Dev is in percentage
    sortino_ratio = (expected_gain - risk_free_return_5_days) / std_dev
    return sortino_ratio

# Apply the function to calculate Sortino Ratio
filtered_df['Sortino_Ratio'] = filtered_df.apply(calculate_sortino, axis=1)

# Display the updated DataFrame
#print(filtered_df)

results_df_sorted = filtered_df.sort_values(by='Expected_Gain', ascending=False)

# Print sorted results_df
print(results_df_sorted)

In [ ]:
selected_tickers = filtered_df['Ticker'].tolist()
print(filtered_df.columns.tolist())